In [4]:
!git clone https://github.com/seyonechithrananda/bert-loves-chemistry.git

Cloning into 'bert-loves-chemistry'...
remote: Enumerating objects: 1566, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1566 (delta 96), reused 92 (delta 91), pack-reused 1364
Receiving objects: 100% (1566/1566), 55.35 MiB | 14.27 MiB/s, done.
Resolving deltas: 100% (1000/1000), done.


In [5]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from bertviz import head_view

model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k", output_attentions=True, output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")


/home/kinnunes/Virtual Laboratories for Pharmaceutical/vlab/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
smiles = 'C1=CC=<mask>=C1'
tokenized = tokenizer.encode(smiles)

In [35]:
encoded_input = tokenizer.encode(smiles, return_tensors="pt")
output = model(encoded_input)
logits = output.logits[0,tokenized.index(4),:]

In [36]:
logits.argmax()

tensor(262)

In [37]:
tokenizer.decode(262)

'CC'

In [20]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>', '<mask>']

In [18]:
tokenizer.mask_token_id

4

In [17]:
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

TypeError: 'bool' object is not subscriptable

In [31]:
smiles_mask = "C1=CC=CC<mask>C1"
smiles = "C1=CC=CC=C1"

masked_smi = fill_mask(smiles_mask)

for smi in masked_smi:
  print(smi)

{'score': 0.9755934476852417, 'token': 33, 'token_str': '=', 'sequence': 'C1=CC=CC=C1'}
{'score': 0.020923908799886703, 'token': 7, 'token_str': '#', 'sequence': 'C1=CC=CC#C1'}
{'score': 0.0007658947724848986, 'token': 21, 'token_str': '1', 'sequence': 'C1=CC=CC1C1'}
{'score': 0.00041297602001577616, 'token': 22, 'token_str': '2', 'sequence': 'C1=CC=CC2C1'}
{'score': 0.00025319133419543505, 'token': 352, 'token_str': '=[', 'sequence': 'C1=CC=CC=[C1'}


In [113]:
smiles = "CCO"
encoded_input = tokenizer.encode(smiles, return_tensors="pt")
output = model(encoded_input)
output

MaskedLMOutput(loss=None, logits=tensor([[[ -7.6558, -14.0462,  -7.4074,  ..., -15.2723, -14.8948, -15.3795],
         [ -7.4523, -11.3929,  -8.8325,  ..., -12.4926, -12.0528, -11.8053],
         [ -7.0814, -13.6186,  -7.5875,  ..., -14.8810, -14.5286, -15.0764]]],
       grad_fn=<ViewBackward0>), hidden_states=(tensor([[[-0.9260,  1.0885, -1.4870,  ...,  1.4126,  0.0499, -0.0160],
         [-0.3525,  0.7792,  0.5669,  ...,  0.0285,  1.4401, -0.3127],
         [-0.4993,  0.2187,  1.3723,  ...,  0.8860,  1.8355, -0.6392]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[-1.7085,  1.2924, -1.1331,  ...,  0.8577,  0.1635, -0.2324],
         [-0.5091,  1.3939,  1.1030,  ..., -0.3351,  2.3625, -0.6673],
         [-1.6204,  0.2253,  0.8990,  ...,  1.1815,  2.7231, -0.9860]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[-2.0472,  0.7284, -0.7592,  ...,  1.0540,  0.6680,  1.3268],
         [-0.4530,  1.4330,  1.2938,  ..., -0.7182,  1.8009,  1.1720],
         [-2.3590,  0.3752

In [114]:
encoded_input

tensor([[  0, 324,   2]])

In [79]:
output.hidden_states[-1].shape

torch.Size([1, 3, 768])

In [80]:
len(output.hidden_states)

7

In [81]:
#O=C(OCC)C
encoded_input

tensor([[  0, 324,   2]])

In [82]:
tokenizer.decode(encoded_input[0])

'<s>CCO</s>'

In [84]:
output.logits.shape

torch.Size([1, 3, 52000])

In [86]:
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")
print(f"Model config: {model.config}")

Tokenizer vocab size: 7924
Model config: RobertaConfig {
  "_name_or_path": "seyonec/PubChem10M_SMILES_BPE_450k",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_attentions": true,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}



In [48]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
cos(output.logits[0][10],output.logits[0][10])

tensor(1.0000, grad_fn=<SumBackward1>)

In [93]:
output.logits[0][0].detach()

tensor([ -7.6558, -14.0462,  -7.4074,  ..., -15.2723, -14.8948, -15.3795])

In [99]:
m = torch.nn.Softmax()
s = m(output.logits[0][0].detach())

/home/kinnunes/Virtual Laboratories for Pharmaceutical/vlab/lib/python3.12/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [102]:
s.argmax()

tensor(21)

In [108]:
m = torch.nn.Softmax()
s = m(output.logits[0][1].detach())
tokenizer.decode(torch.tensor([s.argmax()]))

'CCO'

In [109]:
s[-100:]

tensor([1.3945e-08, 5.1417e-09, 1.0764e-08, 8.4116e-09, 5.9243e-09, 1.1958e-08,
        1.8460e-08, 1.2109e-08, 6.8984e-09, 6.0611e-09, 1.2832e-08, 1.3336e-08,
        5.8392e-09, 4.8676e-09, 6.2577e-09, 1.4798e-08, 6.9734e-09, 8.5084e-09,
        7.6099e-09, 7.6932e-09, 6.5979e-09, 5.2883e-09, 1.2863e-08, 7.8229e-09,
        4.2298e-09, 1.0856e-08, 1.0040e-08, 9.6652e-09, 7.5489e-09, 1.2422e-08,
        4.8839e-09, 1.4095e-08, 1.8170e-08, 4.8783e-09, 9.7936e-09, 1.1018e-08,
        6.2736e-09, 4.8806e-09, 5.7192e-09, 8.1560e-09, 5.9114e-09, 5.2537e-09,
        6.9231e-09, 8.0058e-09, 1.5052e-08, 9.0575e-09, 1.4172e-08, 6.8955e-09,
        1.3890e-08, 1.1775e-08, 5.3803e-09, 1.8080e-08, 6.1643e-09, 7.7689e-09,
        1.2203e-08, 5.3980e-09, 9.4877e-09, 1.0576e-08, 1.0425e-08, 9.5385e-09,
        9.3665e-09, 1.2337e-08, 1.1493e-08, 8.0305e-09, 1.3615e-08, 1.1460e-08,
        5.6252e-09, 5.5747e-09, 4.3661e-09, 1.4847e-08, 9.8487e-09, 7.0689e-09,
        1.0863e-08, 6.5225e-09, 1.0949e-

In [110]:
s[7900:]

tensor([6.8766e-07, 1.3366e-08, 1.7296e-07, 1.3566e-06, 3.0956e-07, 1.3600e-06,
        1.9463e-06, 9.0404e-04, 1.7684e-06, 1.5250e-05, 2.1486e-06, 8.9456e-07,
        2.5004e-05, 3.3969e-03, 7.1444e-07, 1.0598e-04, 1.0077e-06, 4.5854e-05,
        1.3879e-06, 5.3039e-06, 3.3418e-06, 4.2262e-03, 2.1702e-05, 1.6995e-03,
        6.1067e-04, 1.0590e-03, 1.0454e-04, 4.0084e-04, 4.6368e-06, 1.2190e-04,
        6.7041e-05, 1.7494e-06, 1.1451e-06, 1.5270e-04, 3.6365e-06, 1.6737e-06,
        3.9430e-06, 3.0914e-06, 1.0548e-04, 7.5182e-05, 1.1254e-05, 1.4388e-06,
        9.3995e-05, 3.1024e-06, 1.4391e-06, 5.2804e-03, 2.2665e-06, 3.1863e-06,
        1.1972e-06, 9.6301e-07, 2.1825e-04, 2.9617e-04, 6.0474e-04, 7.1826e-07,
        5.9401e-07, 1.4375e-04, 1.0582e-05, 1.1215e-06, 4.8120e-05, 1.4863e-04,
        1.7824e-06, 1.2336e-06, 4.1196e-06, 7.5092e-05, 1.1030e-06, 4.6979e-02,
        2.5599e-06, 2.1039e-06, 9.0069e-07, 1.4240e-06, 2.9356e-06, 2.0591e-04,
        5.6854e-07, 5.2945e-06, 8.7724e-

In [21]:
ethanol = 'C'
ethanol_input = tokenizer.encode(ethanol, return_tensors='pt')
ethanol = model(ethanol_input)

In [22]:
ethanol.logits.shape

torch.Size([1, 3, 52000])

In [99]:
methanol = 'CO'
methanol_input = tokenizer.encode(methanol, return_tensors='pt')
methanol = model(methanol_input)

In [100]:
methanol.logits

tensor([[[ -7.5177, -13.6238,  -8.3360,  ..., -14.6676, -14.1055, -14.3423],
         [ -8.2455,  -9.1575,  -8.5876,  ...,  -9.5612,  -8.8451,  -8.7961],
         [ -7.5205, -12.2304,  -8.2294,  ..., -13.2789, -12.7507, -12.9583]]],
       grad_fn=<ViewBackward0>)

In [7]:
print(tokenizer)

RobertaTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_450k', vocab_size=7924, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


In [93]:
test.mean(axis=1)

tensor([[ -7.7612, -11.6706,  -8.3844,  ..., -12.5026, -11.9004, -12.0322]])

In [102]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
cos(methanol.logits[0].mean(axis=1),ethanol.logits[0].mean(axis=1))

tensor(0.9972, grad_fn=<SumBackward1>)

tensor([ -7.7461, -10.9860,  -8.6876,  ..., -11.3977, -10.7377, -10.8192],
       grad_fn=<SelectBackward0>)

In [111]:
model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k", output_attentions=True, output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")


/home/kinnunes/Virtual Laboratories for Pharmaceutical/vlab/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [112]:
tokenizer

RobertaTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_450k', vocab_size=7924, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

In [116]:
input_  = 'C1=CC=C(C=C1)I.B(C1=CC=CC=C1)(O)O.CO'
output = tokenizer.encode(input_, return_tensors='pt')
encoded = model(output)

In [117]:
output

tensor([[  0,  39,  21,  33, 262,  33,  39,  12,  39,  33,  39,  21,  13,  45,
          18,  38,  12,  39,  21,  33, 262,  33, 262,  33,  39,  21, 274,  51,
          13,  51,  18, 298,   2]])